# Europa Raypath Simulator

In [1]:
# imports
import sys, os
sys.path.append("../../src")

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

os.environ["PATH"] += os.pathsep + '/usr/share/texlive/texmf-dist/tex/xelatex'

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from surface import *
from source import *
from simulator import *
from focus import *

GPU's detected. Enabling CUDA compute


## Specular Point Target

*Using a $3\degree$ wide boxcar*

| Transit | Traces | Altitude | Target Depth | Polarization | f₀ | Power |
|---------|--------|----------|---------------|--------------|----------|-----------|
| 30 km   | 300    | 25 km    | 1 km          | v            | 9 MHz    | 11.75 W   |

| Subsurface Gain | Surface Gain | Rx Opening | Rx Window | Rx Sampling | ε₁ | ε₂  | σ₁ | σ₂       |
|------------------|---------------|------------|------------|--------------|----|-----|-----|-----------|
| 80 dB            | 76 dB         | 22.5 km      | 10 km      | 48 MHz       | 1  | 3.15 | 0 S | 10⁻⁶ S   |


In [2]:
# create surface
surf = Surface(origin=(4000, 4000), dims=(400, 400), fs=5, overlap=0)
surf.gen_flat(0)

In [3]:
# generate linspace of sources
ss = source_linspace('x', -10e3, 20e3, 5050, 25e3, 500, 9e6, 1e6)

# target location
t = (5000, 5050, -1000)   # target location (x, y, z)

par = {
    'surf_gain':77,
    'rx_window_offset':22.5e3,
    'rx_window_m':10e3,
    'aspect':0.5
}

In [4]:
par['surf_gain'] = 76
rdrgrm, time = run_sim_ms(surf, ss, t, reflect=True, polarization='v', sltrng=False, pt_response="gaussian", refl_center=True, show=False, par=par, nsmpl=1601)

## Focus

In [5]:
sx = [s.coord[0] for s in ss]
sz = ss[0].coord[2]
c1 = 299792458
c2 = c1 / np.sqrt(3.15)

sampl = 48e6   # sampling rate [MHz]
wavlen = 33.3  # wavelength    [m]

rb = int((par['rx_window_m'] / c1) / (1 / sampl))
dm = c1 / (sampl)

In [6]:
sltrng   = est_slant_range(sx, sz, t[0], t[2], c1, c2)
sltrng_t = 2 * 10**6 * sltrng / c1
slt_rb   = ((sltrng - par['rx_window_offset']) // dm).astype(int)

In [7]:
k = (2 * np.pi) / wavlen
match_filter = np.exp(-2j * k * sltrng)

In [8]:
focused = focus_jit(rdrgrm, slt_rb, match_filter, rb)

## Generate Figure

In [11]:
extent = (10, 90, 2*((par['rx_window_offset'] + par['rx_window_m'])/c1)*10**6, 2*(par['rx_window_offset']/c1)*10**6)
fig, ax = plt.subplots(2, 1, figsize=(4, 4), constrained_layout=True, dpi=300, sharex=True)

im0 = ax[0].imshow(np.abs(rdrgrm)*1e6, cmap="gray", aspect=par['aspect']*1.5, extent=extent)
im1 = ax[1].imshow(np.abs(focused)*1e6, cmap="gray", aspect=par['aspect']*1.5, extent=extent)

labels = ["(a)", "(b)"]
for a, label in zip(ax, labels):
    a.set_ylabel("Range [µs]", fontsize=11)
    a.tick_params(axis="both", which="major", labelsize=9, direction="out")
    a.tick_params(axis="both", which="minor", direction="out")
    a.text(0.02, 0.95, label, transform=a.transAxes, fontsize=11,
           fontweight="bold", va="top", ha="left", color="black",
           bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=2))

ax[1].set_xlabel("Azimuth [km]", fontsize=11)

cbar = fig.colorbar(im1, ax=ax, orientation="vertical", fraction=0.05, pad=0.04)
cbar.set_label("Power [µW]", fontsize=11)
cbar.ax.tick_params(labelsize=9)

plt.savefig("focused.pgf", dpi=300, bbox_inches="tight")
plt.close()
